<center><img style="width:500px;" src="https://s3.amazonaws.com/thinkific-import/370184/U9zyVFboQScufA7Iih3p_Capture_d_e_cran_2020_09_23_a__00_11_29_png"></center>

# Parcours : Le Data Mining pour tous

## Niveau 2 : Passez d'Excel à Python 

### Cas pratique : Les données COVID-19

#### Etape 0 : Initialisation du notebook

In [2]:
# Import des modules

import pandas as pd
from datetime import datetime, timedelta, date

#### Etape 1 : Récupération des données sources

Source COVID-19 (John Hopkins University)<br>
Données mise à jour tous les jours entre 2h et 8h (CET)<br> 
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

In [20]:
# Récupérer un fichier sur les données COVID
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(url)
# df.columns = map(lambda x: str(x).upper(), df.columns)
df.tail(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20
262,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,41957,42432,42840,43256,43664,43945,44299,44684,45200,45658
263,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
264,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2041,2047,2049,2050,2051,2051,2052,2052,2053,2053
265,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,15089,15170,15224,15301,15339,15415,15458,15549,15587,15616
266,NaN,Zimbabwe,-19.015438,29.154857,0,0,0,0,0,0,...,7898,7915,7919,7951,7994,8010,8011,8021,8036,8055


In [21]:
print(df.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '10/5/20', '10/6/20', '10/7/20', '10/8/20', '10/9/20', '10/10/20',
       '10/11/20', '10/12/20', '10/13/20', '10/14/20'],
      dtype='object', length=271)


In [22]:
# Url du répertoire github
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'

# Liste des fichiers à récupérer sur le répertoire
files = [
         'time_series_covid19_confirmed_global.csv',
         'time_series_covid19_deaths_global.csv',
         'time_series_covid19_recovered_global.csv'
        ]

# Boucle pour récupérer et concaténer les fichier sources
data_source = pd.DataFrame()
for file in files:
    df = pd.read_csv(f'{url}{file}')
    df['Type'] = file[20:-11].capitalize()
    data_source = pd.concat([data_source, df], axis=0)
    
data_source.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,Type
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39486,39548,39616,39693,39703,39799,39870,39928,39994,Confirmed
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,14568,14730,14899,15066,15231,15399,15570,15752,15955,Confirmed
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,52399,52520,52658,52804,52940,53072,53325,53399,53584,Confirmed
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,2370,2568,2568,2696,2696,2696,2995,2995,3190,Confirmed
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,5725,5725,5958,6031,6246,6366,6488,6680,6846,Confirmed


#### Etape 2 - Mise au format base de donnée

In [63]:
# Initialisation du dataframe
data_pivot = data_source.copy()
data_pivot.to_csv('DONNEES_INIT.csv',
                    sep=";",
                    decimal=",",
                    index=False)


# Colonnes à supprimer
to_del = ['Province/State', 'Lat', 'Long']

# Suppression des colonnes inutiles
data_pivot = data_pivot.drop(to_del, axis=1)


# Colonnes à conserver
to_keep = ['Type', 'Country/Region']

# Pivot des colonnes
data_pivot = data_pivot.melt(id_vars=to_keep, 
                             value_name='Value', 
                             var_name='Date')



# Colonnes à aggréger
to_group = ['Date', 'Country/Region', 'Type']

# Regroupement et aggrégation
data_pivot['Date'] = pd.to_datetime(data_pivot['Date'], format='%m/%d/%y')

data_pivot = data_pivot.groupby(to_group, as_index=False).agg({'Value': 'sum'})
data_pivot.head(5)

# criteras = data_pivot[['Country/Region','Type']].drop_duplicates()
# df_ma = pd.DataFrame()
# for _, critera in criteras.iterrows():
#     country = critera[0]
#     typo = critera[1]
#     tmp_df = data_pivot.copy()
#     tmp_df = tmp_df[(tmp_df['Country/Region'] == country) & 
#                     (tmp_df['Type'] == typo)]
#     tmp_df['MA'] = tmp_df['Value'].rolling(window=7).mean()
#     df_ma = pd.concat([df_ma, tmp_df], axis=0)

# df_ma = df_ma[(df_ma['Country/Region'] == 'France') & (df_ma['Type'] == 'Confirmed')]
# df_ma = df_ma.sort_values(by='Date', ascending=False)
# df_ma.head(7)

,Date,Country/Region,Type,Value
0,2020-01-22,Afghanistan,Confirmed,0
1,2020-01-22,Afghanistan,Deaths,0
2,2020-01-22,Afghanistan,Recovered,0
3,2020-01-22,Albania,Confirmed,0
4,2020-01-22,Albania,Deaths,0


#### Etape 3 - Nettoyage de la base

In [64]:
# Initialisation du dataframe
data_clean = data_pivot.copy()

# Mise au format de la date
data_clean['Date'] = pd.to_datetime(data_clean['Date'], format='%m/%d/%y')

# Renomage de la colonne 'Close' en 'VALUE'
data_clean = data_clean.rename(columns={'Country/Region': 'Country'})

# Format des colonnes en majuscule
data_clean.columns = map(lambda x: str(x).upper(), data_clean.columns)
    
data_clean.head(5)

,DATE,COUNTRY,TYPE,VALUE
0,2020-01-22,Afghanistan,Confirmed,0
1,2020-01-22,Afghanistan,Deaths,0
2,2020-01-22,Afghanistan,Recovered,0
3,2020-01-22,Albania,Confirmed,0
4,2020-01-22,Albania,Deaths,0


#### Etape 4 - Enrichissement des données

4.1 - AJOUT DES CONTINENTS

In [65]:
# Initialisation du dataframe
data_enr = data_clean.copy()

# Lecture du fichier REF_CONTINENT
ref_continent = pd.read_excel('REF_CS.xlsx',
                              sheet_name='REF_CONTINENT')

# Jointure avec le dataframe principal
data_enr = pd.merge(data_enr, 
                    ref_continent, 
                    on=['COUNTRY'],
                    how='left'
                   )

# Nettoyage et affectation d'une valeur pour les valeurs nulles
data_enr['CONTINENT'] = data_enr['CONTINENT'].fillna(0)
data_enr.loc[data_enr['CONTINENT'] == 0, 'CONTINENT'] = 'To be affected'

data_enr.head(5)

,DATE,COUNTRY,TYPE,VALUE,CONTINENT
0,2020-01-22,Afghanistan,Confirmed,0,Asia
1,2020-01-22,Afghanistan,Deaths,0,Asia
2,2020-01-22,Afghanistan,Recovered,0,Asia
3,2020-01-22,Albania,Confirmed,0,Europe
4,2020-01-22,Albania,Deaths,0,Europe


4.2 - CALCUL DES CAS ACTIFS

In [66]:
# Calcul des cas actifs
data_active = data_enr.copy()

data_active.loc[data_active['TYPE'] != 'Confirmed', 'VALUE'] = data_active['VALUE'] * (-1)
data_active.loc[:, 'TYPE'] = 'Active Cases'

# Colonnes à grouper
to_group = ['DATE', 'COUNTRY', 'CONTINENT', 'TYPE']

# Colonnes à aggréger 
to_agg = {'VALUE': 'sum'}

# Regroupement et aggrégation
data_active = data_active.groupby(to_group, as_index=False).agg(to_agg)
data_active.head(5)

,DATE,COUNTRY,CONTINENT,TYPE,VALUE
0,2020-01-22,Afghanistan,Asia,Active Cases,0
1,2020-01-22,Albania,Europe,Active Cases,0
2,2020-01-22,Algeria,Africa,Active Cases,0
3,2020-01-22,Andorra,Europe,Active Cases,0
4,2020-01-22,Angola,Africa,Active Cases,0


4.3 - CONCATENATION DES 2 BASES DE DONNEES + CALCUL DES VARIATIONS

In [67]:
# Initialisation du dataframe
data_bdd = data_enr.copy()

# Concat
data_bdd = pd.concat([data_enr, data_active], axis=0)

data_bdd.head(5)

,DATE,COUNTRY,TYPE,VALUE,CONTINENT
0,2020-01-22,Afghanistan,Confirmed,0,Asia
1,2020-01-22,Afghanistan,Deaths,0,Asia
2,2020-01-22,Afghanistan,Recovered,0,Asia
3,2020-01-22,Albania,Confirmed,0,Europe
4,2020-01-22,Albania,Deaths,0,Europe


In [74]:
data_var = data_bdd.copy()

criteras = data_var[['COUNTRY','TYPE']].drop_duplicates()
data_final = pd.DataFrame()

for _, critera in criteras.iterrows():
    country = critera[0]
    typo = critera[1]
    tmp_df = data_var.copy()
    tmp_df = tmp_df[(tmp_df['COUNTRY'] == country) & 
                    (tmp_df['TYPE'] == typo)].reset_index(drop=True)
    
    for i in range(1, len(tmp_df)):
        print(i)
        value_last = tmp_df.loc[i-1, 'VALUE']
        tmp_df['VALUE_LAST'] = value_last
        print(tmp_df)
        break
# #     tmp_df['MA'] = tmp_df['Value'].rolling(window=7).mean()
# #     df_ma = pd.concat([df_ma, tmp_df], axis=0)

# for i in range(1, len(data_var)):
#     value_last = data_var.loc[i-1, 'VALUE']
#     value = data_var.loc[i, 'VALUE']
#     print(value_last, value)
#     break

1
          DATE      COUNTRY       TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Afghanistan  Confirmed      0      Asia           0
1   2020-01-23  Afghanistan  Confirmed      0      Asia           0
2   2020-01-24  Afghanistan  Confirmed      0      Asia           0
3   2020-01-25  Afghanistan  Confirmed      0      Asia           0
4   2020-01-26  Afghanistan  Confirmed      0      Asia           0
..         ...          ...        ...    ...       ...         ...
262 2020-10-10  Afghanistan  Confirmed  39703      Asia           0
263 2020-10-11  Afghanistan  Confirmed  39799      Asia           0
264 2020-10-12  Afghanistan  Confirmed  39870      Asia           0
265 2020-10-13  Afghanistan  Confirmed  39928      Asia           0
266 2020-10-14  Afghanistan  Confirmed  39994      Asia           0

[267 rows x 6 columns]
1
          DATE      COUNTRY    TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Afghanistan  Deaths      0      Asia           0
1   2020-01-23  Afghanista

1
          DATE              COUNTRY       TYPE  VALUE      CONTINENT  \
0   2020-01-22  Antigua and Barbuda  Confirmed      0  North America   
1   2020-01-23  Antigua and Barbuda  Confirmed      0  North America   
2   2020-01-24  Antigua and Barbuda  Confirmed      0  North America   
3   2020-01-25  Antigua and Barbuda  Confirmed      0  North America   
4   2020-01-26  Antigua and Barbuda  Confirmed      0  North America   
..         ...                  ...        ...    ...            ...   
262 2020-10-10  Antigua and Barbuda  Confirmed    111  North America   
263 2020-10-11  Antigua and Barbuda  Confirmed    111  North America   
264 2020-10-12  Antigua and Barbuda  Confirmed    111  North America   
265 2020-10-13  Antigua and Barbuda  Confirmed    111  North America   
266 2020-10-14  Antigua and Barbuda  Confirmed    112  North America   

     VALUE_LAST  
0             0  
1             0  
2             0  
3             0  
4             0  
..          ...  
262    

1
          DATE  COUNTRY    TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Austria  Deaths      0    Europe           0
1   2020-01-23  Austria  Deaths      0    Europe           0
2   2020-01-24  Austria  Deaths      0    Europe           0
3   2020-01-25  Austria  Deaths      0    Europe           0
4   2020-01-26  Austria  Deaths      0    Europe           0
..         ...      ...     ...    ...       ...         ...
262 2020-10-10  Austria  Deaths    852    Europe           0
263 2020-10-11  Austria  Deaths    851    Europe           0
264 2020-10-12  Austria  Deaths    855    Europe           0
265 2020-10-13  Austria  Deaths    861    Europe           0
266 2020-10-14  Austria  Deaths    872    Europe           0

[267 rows x 6 columns]
1
          DATE  COUNTRY       TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Austria  Recovered      0    Europe           0
1   2020-01-23  Austria  Recovered      0    Europe           0
2   2020-01-24  Austria  Recovered      0    Eur

1
          DATE     COUNTRY       TYPE   VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Bangladesh  Recovered       0      Asia           0
1   2020-01-23  Bangladesh  Recovered       0      Asia           0
2   2020-01-24  Bangladesh  Recovered       0      Asia           0
3   2020-01-25  Bangladesh  Recovered       0      Asia           0
4   2020-01-26  Bangladesh  Recovered       0      Asia           0
..         ...         ...        ...     ...       ...         ...
262 2020-10-10  Bangladesh  Recovered  291365      Asia           0
263 2020-10-11  Bangladesh  Recovered  292860      Asia           0
264 2020-10-12  Bangladesh  Recovered  294391      Asia           0
265 2020-10-13  Bangladesh  Recovered  295873      Asia           0
266 2020-10-14  Bangladesh  Recovered  297449      Asia           0

[267 rows x 6 columns]
1
          DATE   COUNTRY       TYPE  VALUE      CONTINENT  VALUE_LAST
0   2020-01-22  Barbados  Confirmed      0  North America           0
1   2020-01-23  

1
          DATE COUNTRY       TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22   Benin  Confirmed      0    Africa           0
1   2020-01-23   Benin  Confirmed      0    Africa           0
2   2020-01-24   Benin  Confirmed      0    Africa           0
3   2020-01-25   Benin  Confirmed      0    Africa           0
4   2020-01-26   Benin  Confirmed      0    Africa           0
..         ...     ...        ...    ...       ...         ...
262 2020-10-10   Benin  Confirmed   2411    Africa           0
263 2020-10-11   Benin  Confirmed   2411    Africa           0
264 2020-10-12   Benin  Confirmed   2411    Africa           0
265 2020-10-13   Benin  Confirmed   2411    Africa           0
266 2020-10-14   Benin  Confirmed   2478    Africa           0

[267 rows x 6 columns]
1
          DATE COUNTRY    TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22   Benin  Deaths      0    Africa           0
1   2020-01-23   Benin  Deaths      0    Africa           0
2   2020-01-24   Benin  Deaths      

...

          DATE      COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Philippines  Active Cases      0      Asia           0
1   2020-01-23  Philippines  Active Cases      0      Asia           0
2   2020-01-24  Philippines  Active Cases      0      Asia           0
3   2020-01-25  Philippines  Active Cases      0      Asia           0
4   2020-01-26  Philippines  Active Cases      0      Asia           0
..         ...          ...           ...    ...       ...         ...
262 2020-10-10  Philippines  Active Cases  54594      Asia           0
263 2020-10-11  Philippines  Active Cases  39945      Asia           0
264 2020-10-12  Philippines  Active Cases  43332      Asia           0
265 2020-10-13  Philippines  Active Cases  44958      Asia           0
266 2020-10-14  Philippines  Active Cases  46227      Asia           0

[267 rows x 6 columns]
1
          DATE COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Poland  Active Cases      0    Europe   

          DATE                COUNTRY          TYPE  VALUE CONTINENT  \
0   2020-01-22  Sao Tome and Principe  Active Cases      0    Africa   
1   2020-01-23  Sao Tome and Principe  Active Cases      0    Africa   
2   2020-01-24  Sao Tome and Principe  Active Cases      0    Africa   
3   2020-01-25  Sao Tome and Principe  Active Cases      0    Africa   
4   2020-01-26  Sao Tome and Principe  Active Cases      0    Africa   
..         ...                    ...           ...    ...       ...   
262 2020-10-10  Sao Tome and Principe  Active Cases     17    Africa   
263 2020-10-11  Sao Tome and Principe  Active Cases     22    Africa   
264 2020-10-12  Sao Tome and Principe  Active Cases     19    Africa   
265 2020-10-13  Sao Tome and Principe  Active Cases     19    Africa   
266 2020-10-14  Sao Tome and Principe  Active Cases     18    Africa   

     VALUE_LAST  
0             0  
1             0  
2             0  
3             0  
4             0  
..          ...  
262      

1
          DATE      COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  South Sudan  Active Cases      0    Africa           0
1   2020-01-23  South Sudan  Active Cases      0    Africa           0
2   2020-01-24  South Sudan  Active Cases      0    Africa           0
3   2020-01-25  South Sudan  Active Cases      0    Africa           0
4   2020-01-26  South Sudan  Active Cases      0    Africa           0
..         ...          ...           ...    ...       ...         ...
262 2020-10-10  South Sudan  Active Cases   1417    Africa           0
263 2020-10-11  South Sudan  Active Cases   1432    Africa           0
264 2020-10-12  South Sudan  Active Cases   1442    Africa           0
265 2020-10-13  South Sudan  Active Cases   1453    Africa           0
266 2020-10-14  South Sudan  Active Cases   1453    Africa           0

[267 rows x 6 columns]
1
          DATE COUNTRY          TYPE   VALUE CONTINENT  VALUE_LAST
0   2020-01-22   Spain  Active Cases       0    Europ

1
          DATE   COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Tanzania  Active Cases      0    Africa           0
1   2020-01-23  Tanzania  Active Cases      0    Africa           0
2   2020-01-24  Tanzania  Active Cases      0    Africa           0
3   2020-01-25  Tanzania  Active Cases      0    Africa           0
4   2020-01-26  Tanzania  Active Cases      0    Africa           0
..         ...       ...           ...    ...       ...         ...
262 2020-10-10  Tanzania  Active Cases    305    Africa           0
263 2020-10-11  Tanzania  Active Cases    305    Africa           0
264 2020-10-12  Tanzania  Active Cases    305    Africa           0
265 2020-10-13  Tanzania  Active Cases    305    Africa           0
266 2020-10-14  Tanzania  Active Cases    305    Africa           0

[267 rows x 6 columns]
1
          DATE   COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Thailand  Active Cases      2      Asia           2
1   2020-01-23  Thai

1
          DATE  COUNTRY          TYPE  VALUE      CONTINENT  VALUE_LAST
0   2020-01-22  Uruguay  Active Cases      0  South America           0
1   2020-01-23  Uruguay  Active Cases      0  South America           0
2   2020-01-24  Uruguay  Active Cases      0  South America           0
3   2020-01-25  Uruguay  Active Cases      0  South America           0
4   2020-01-26  Uruguay  Active Cases      0  South America           0
..         ...      ...           ...    ...            ...         ...
262 2020-10-10  Uruguay  Active Cases    288  South America           0
263 2020-10-11  Uruguay  Active Cases    302  South America           0
264 2020-10-12  Uruguay  Active Cases    312  South America           0
265 2020-10-13  Uruguay  Active Cases    299  South America           0
266 2020-10-14  Uruguay  Active Cases    330  South America           0

[267 rows x 6 columns]
1
          DATE     COUNTRY          TYPE  VALUE CONTINENT  VALUE_LAST
0   2020-01-22  Uzbekistan  Active Cas

#### Etape 5 : Sauvegarde et exposition du csv

In [9]:
# Ajout de la date de mise à jour
data_bdd['DATE_MAJ'] = datetime.now().strftime('%Y/%m/%d %H:%M:%S')

# Sauvegarde des données au format csv
data_bdd.to_csv('BASE_COVID.csv',
                sep=";",
                decimal=",",
                index=False)

print('CSV sauvegardé dans le répertoire courant.')

CSV sauvegardé dans le répertoire courant.
